<a href="https://colab.research.google.com/github/HARDIK218/Large-Language-Model/blob/main/Geeta_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.15
!pip install torch==2.2.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.2
    Uninstalling torch-2.2.2:
      Successfully uninstalled torch-2.2.2


In [ ]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [ ]:
#################################################################
# Tokenizer
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.2'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)


In [ ]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
!playwright install
!playwright install-deps

155.3 MiB [] 0% 0.0s155.3 MiB [] 0% 4.4s155.3 MiB [] 0% 4.3s155.3 MiB [] 1% 3.7s155.3 MiB [] 1% 3.4s155.3 MiB [] 2% 3.3s155.3 MiB [] 2% 3.5s155.3 MiB [] 3% 3.6s155.3 MiB [] 3% 3.4s155.3 MiB [] 4% 3.6s155.3 MiB [] 4% 3.7s155.3 MiB [] 5% 3.5s155.3 MiB [] 5% 3.4s155.3 MiB [] 6% 3.3s155.3 MiB [] 7% 3.2s155.3 MiB [] 7% 3.0s155.3 MiB [] 8% 3.0s155.3 MiB [] 9% 3.0s155.3 MiB [] 10% 2.9s155.3 MiB [] 11% 2.9s155.3 MiB [] 12% 2.8s155.3 MiB [] 12% 2.9s155.3 MiB [] 13% 2.8s155.3 MiB [] 14% 2.7s155.3 MiB [] 14% 2.8s155.3 MiB [] 15% 2.7s155.3 MiB [] 16% 2.7s155.3 MiB [] 17% 2.6s155.3 MiB [] 18% 2.6s155.3 MiB [] 19% 2.5s155.3 MiB [] 20% 2.4s155.3 MiB [] 21% 2.3s155.3 MiB [] 23% 2.2s155.3 MiB [] 24% 2.1s155.3 MiB [] 25% 2.0s155.3 MiB [] 26% 1.9s155.3 MiB [] 27% 1.9s155.3 MiB [] 27% 2.0s155.3 MiB [] 28% 1.9s155.3 MiB [] 29% 1.8s155.3 MiB [] 30% 1.8s155.3 MiB [] 31% 1.7s155.3 MiB [] 32% 1.7s155.3 MiB [] 33% 1.7s155.3 MiB [] 34% 1.6s155.3 MiB [] 36% 1.5s155.3 MiB [] 37% 1.5s155.3 MiB [] 38% 1.4s155.3 MiB 

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from html2text import HTML2Text

# Replace 'your_file.csv' with the actual path to your CSV file
csv_file_path = '/content/geetacsv.csv'

# Replace 'your_column_name' with the name of the column you want to extract
column_name = 'Interpretation'

# Read the first 500 rows of the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, nrows=500)

# Create a list to store transformed content
docs_transformed = []

# Create an HTML2Text object
html2text = HTML2Text()

# Extract data from the specified column into a list
for index, row in df.iterrows():
    html_content = row[column_name]
    soup = BeautifulSoup(html_content, 'html.parser')
    plain_text = html2text.handle(str(soup))
    docs_transformed.append(plain_text)

class PageContentWrapper:
    def __init__(self, page_content, metadata={}):
        self.page_content = page_content
        self.metadata = metadata

# Assuming plain_text is the content you want to chunk
docs_transformed_wrapped = [PageContentWrapper(content) for content in docs_transformed]

# Now use docs_transformed_wrapped with CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed_wrapped)



In [ ]:
db = FAISS.from_documents(chunked_documents,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Create prompt template
prompt_template = """
### [INST] Instruction: Answer the question based on your docs knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)


rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("i accidently hurt my friend and iam very sad about it what should i do")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
result['context']

[Document(page_content='Arjuna humbly seeks forgiveness from Lord Krishna, realizing that in his\naffectionate friendship, he may have addressed Krishna casually without\nrecognizing His divine stature.'),
 Document(page_content="Arjuna seeks forgiveness from Lord Krishna for any irreverent actions or\nremarks made in jest or play, whether knowingly or unknowingly, recognizing\nKrishna's immeasurable nature."),
 Document(page_content='Arjuna expresses deep remorse at the thought of committing a great sin by\nkilling his relatives, driven by the desire for the pleasures of rulership.'),
 Document(page_content='Yoga destroys sorrow for the one who is moderate in eating, recreation, work,\nsleep, and wakefulness.')]

In [ ]:
text = result['text']

# Add line breaks to format the text as a paragraph
formatted_text = text.replace('\n', ' ')  # Replace existing line breaks with spaces
formatted_text = formatted_text.replace('. ', '.\n\n')  # Add double line breaks after periods

# Print the formatted text
print(formatted_text)

 ### [INST] Instruction: Answer the question based on your docs knowledge.

Here is context to help:  [Document(page_content='Arjuna humbly seeks forgiveness from Lord Krishna, realizing that in his\naffectionate friendship, he may have addressed Krishna casually without\nrecognizing His divine stature.'), Document(page_content="Arjuna seeks forgiveness from Lord Krishna for any irreverent actions or\nremarks made in jest or play, whether knowingly or unknowingly, recognizing\nKrishna's immeasurable nature."), Document(page_content='Arjuna expresses deep remorse at the thought of committing a great sin by\nkilling his relatives, driven by the desire for the pleasures of rulership.'), Document(page_content='Yoga destroys sorrow for the one who is moderate in eating, recreation, work,\nsleep, and wakefulness.')]  ### QUESTION: i accidently hurt my friend and iam very sad about it what should i do [/INST]   Based on the provided documents, Arjuna's actions involve seeking forgiveness from